In [1]:
import json
import pandas as pd

In [25]:
df = pd.read_csv('benzinga.csv')
newsfeeds = df.to_json(orient='records')
newsfeeds = json.loads(newsfeeds)
len(newsfeeds)

14072

In [4]:
from gensim.models import KeyedVectors
from simhash import Simhash, SimhashIndex
import numpy as np

In [5]:
# omit the loeading messages, unlike the class exercise
model_w2v_AP = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [6]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [27]:
# Get a subset of 100 Webhose titles assign index values
feeds = []
i = 0
for feed in newsfeeds[:]:
    feed['id'] = i
    #print(feed['id'], str(feed['title']))
    i += 1
    feeds.append(feed)

In [28]:
import logging
logging.getLogger('simhash').setLevel(logging.CRITICAL)

hamming_distance = 10
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=hamming_distance)

In [29]:
# this list keeps the first occurance of duplicated titles
list_dupi = set()
i = 0

# loop to find all the duplicate
while i < len(feeds):
    feed_hash = Simhash(str(feeds[i]['title']))
    dup_indices = index.get_near_dups(feed_hash)
    if dup_indices:
        dup_indices = sorted(set([int(ind) for ind in dup_indices]))
        list_dupi.update( dup_indices[1:] )
    i+=1
    if i in list_dupi:
        i+=1

In [30]:
len(list_dupi)

1130

In [31]:
feeds = [feed for feed in feeds if int(feed['id']) not in list_dupi]
print(len(feeds))

12942


In [32]:
file_path = "benzinga.json"

with open(file_path, "w") as file:
    json.dump(feeds, file)